In [59]:
import os

import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
from functools import partial
from changepoynt.algorithms.base_algorithm import Algorithm
from changepoynt.utils import normalization
from changepoynt.utils import linalg as lg
import fbpca
import multiprocessing as mp
import numba as nb


In [63]:
def left_entropy(hankel: np.ndarray, rank: int, random_rank: int, method: str, threads: int) -> float:
    """
    Entropy Singular Spectrum Transformation.

    :param hankel: the hankel matrix of the signal
    :param rank: the number of (approximated) eigenvectors as subspace of H1
    :param random_rank: the sampling parameter for the randomized svd
    :param method: which rsvd method to use
    :param threads: the numba of threads numba is allowed to use
    :return: the change point score, the input vector x0
    """
    # compute the left and right eigenvectors using the randomized svd for fast computation
    threads_before = nb.get_num_threads()
    nb.set_num_threads(threads)
    if method == 'fbrsvd':
        right_eigenvectors, eigenvalues, left_eigenvectors = fbpca.pca(hankel, rank, True)
    elif method == 'rsvd':
        right_eigenvectors, eigenvalues, left_eigenvectors = lg.randomized_hankel_svd(hankel, rank,
                                                                                      oversampling_p=random_rank - rank)
    else:
        raise NotImplementedError(f'Method {method} is not available.')
    nb.set_num_threads(threads_before)

    # shift the left eigenvectors up
    left_eigenvectors = left_eigenvectors - np.min(left_eigenvectors, axis=1)[:, None] + 1

    # make the eigenvectors into "probability distributions" so their sum of elements is one
    left_eigenvectors = left_eigenvectors/np.sum(left_eigenvectors, axis=1)[:, None]

    # compute the normalized entropy of the eigenvectors
    """entropy = (-1 *
               np.sum(np.log2(left_eigenvectors, out=np.zeros_like(left_eigenvectors), where=(left_eigenvectors != 0))
                      * left_eigenvectors, axis=1))/np.log2(hankel.shape[1])"""
    # skew = np.abs(skewness(left_eigenvectors, np.tile(np.linspace(0, hankel.shape[1]-1, hankel.shape[1]), (rank, 1))))
    skew = np.abs(left_right_diff(left_eigenvectors))

    # compute the weighted mean of the entropy
    # weighted_entropy = (eigenvalues @ ((1-entropy)*skew))/np.sum(eigenvalues)
    weighted_entropy = (eigenvalues @ skew) / np.sum(eigenvalues)

    return weighted_entropy

def left_right_diff(left_eigenvectors: np.ndarray):
    n = left_eigenvectors.shape[1]//2
    return np.mean(left_eigenvectors[:, :n]-left_eigenvectors[:, n:], axis=1)

    # some plotting utilities
def plot_data_and_score(raw_data, score):
    f, ax = plt.subplots(2,1,figsize=(20,10))
    ax[0].plot(raw_data); ax[0].set_title("time series")
    ax[1].plot(score,"r"); ax[1].set_title("change score")
    x_grid, y_grid = np.meshgrid(np.arange(len(score)),np.linspace(*ax[0].get_ylim()))
    z_grid = (score/np.max(score))[x_grid]
    ax[0].contourf(x_grid, y_grid, z_grid,alpha=0.5, cmap="BuPu")

In [61]:
# import dataset
time_series = np.loadtxt("DJ_block2_layer1_hole1_torque.csv", delimiter=",", dtype=float)

In [64]:
# Hardcode variables 
window_length = 150
n_windows = window_length//2
rank = 5
scale = True
random_rank = min(rank + 10, window_length, n_windows)
lag = n_windows
scoring_step = 1
parallel = False
use_fast_hankel = False
method = 'fbrsvd'
threads = os.cpu_count()//2
hankel_function = lg.compile_hankel


# check the dimensions of the input array

assert time_series.ndim == 1, "Time series needs to be an 1D array."

# check that we have at least two windows

assert time_series.shape[0] > window_length, 'Time series needs to be longer than window length.'

# compute the starting point of the scoring (past and future hankel need to fit)
starting_point = window_length + n_windows + lag
assert starting_point < time_series.shape[0], "The time series is too short to score any points."

# for when "online"
# for x in time_series:

# initialize a scoring array with no values yet
score = np.zeros_like(time_series)

# compute the offset
offset = (n_windows + lag)

for idx in range(starting_point, time_series.shape[0], scoring_step):

    # compile the past hankel matrix (H1)
    hankel_past = hankel_function(time_series, idx - lag, window_length, n_windows)

    # compile the future hankel matrix (H2)
    hankel_future = hankel_function(time_series, idx, window_length, n_windows)

    # compute the score and save the returned feedback vector
    score[idx-offset-scoring_step//2:idx-offset+(scoring_step+1)//2] = \
         left_entropy(np.concatenate((hankel_past, hankel_future), axis=1), rank, random_rank, method, threads)

# next steps, have it print values 



In [ ]:

# x = 400
# while x < len(time_series):
#     # print("time series shape",time_series[x-x:x].shape[0])
#     # print("window length",detector.window_length)
#     # print("starting point", detector.window_length + detector.n_windows + detector.lag)
#     score = transform(time_series[x-x:x])
#     plot_data_and_score(time_series[x-x:x],score)
#     x= x+40